# Part 1: Decision Trees

## Introduction

We can quickly use the sklearn library to build and use decision trees. For example,

```
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3)
clf = clf.fit(X, y)
```
<img src="https://upload.wikimedia.org/wikipedia/commons/7/78/Petal-sepal.jpg" width="200" align="right" />

However this lab focuses on learning how the decision trees and random forests are built, by writing our own versions of these tools. In the real world you would be using a library, but the aim here is understanding the fundamentals of how these tools work.

This week I've included the answers to the exercises at the end of the document. Please still try to work out the answers yourself!

We will use as our purity metric the conditional entropy (as covered in the lecture).

We will use a discretised (binarised) version of the <a href="https://en.wikipedia.org/wiki/Iris_flower_data_set">iris dataset</a>. I've converted each of the continuous features to a binary feature, e.g. petal length is now 'long petal' (true/false). The labels are still three categories, either 'setosa', 'versicolor', 'virginica'. The task is to classify the species of Iris from four features (petal length, petal width, sepal length and sepal width).

I've tried to visualise this a little by plotting a bar graph for each class, with the number of rows for each feature being true. For example none of the 50 rows of the 'setosa' species have long sepals, while 88% of the rows of the virginica speices have long sepals.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

X = pd.read_csv('https://drive.usercontent.google.com/download?id=1SHpvd9kee9eiAeviGfOgKR3SdOGfz8yM&export=download&authuser=0',index_col=0)
y = pd.read_csv('https://drive.usercontent.google.com/download?id=1SicnjGCccMnOETeAVA98iCq74cc8_OyD&export=download&authuser=0',index_col=0)
for i,v in enumerate(y['class'].unique()):
  plt.subplot(1,3,i+1)
  plt.title(v)
  print(X[y['class']==v].mean())
  X[y['class']==v].mean().plot.bar()
  plt.ylim([0,1])
  if i>0: plt.yticks([],[])

## Entropy

We need to compute the entropy first.

---


### Exercise 1.1

Fill in the rest of this function with code to compute the entropy of the `ydata` column:

In [ ]:
import numpy as np

def compute_entropy(ydata):
  """
  Returns the entropy of the labels in ydata (a pandas series) in bits.
  """
  #answer here
  return #?

---

In [ ]:
#To test you have coded this correctly, this synthetic test data should lead to
#an estimate of entropy of about 1.72957 bits
#as there are 6 red, 3 blue, 2 yellow, 1 orange. We can assume their probabilities,
#and from that compute the entropy:
#-((6/12)*np.log2(6/12) + (3/12)*np.log2(3/12) + (2/12)*np.log2(2/12) + (1/12)*np.log2(1/12))
testlabels = pd.DataFrame(['red','blue','blue','blue','red','red','yellow','red','red','red','yellow','orange'])
assert abs(compute_entropy(testlabels)-1.72957)<0.0001

#check your method computes the entropy correctly! The assertion will raise an
#AssertionError if it's wrong!

In [ ]:
# We can compute the entropy of the Iris labels:
compute_entropy(y)

## Conditional Entropy

In the lecture, I wrote that the conditional entropy was,

$$-\sum\limits_{x \in X} \sum\limits_{y \in Y}p(x,y) \log_2 p(y|x)$$

but it will be slightly easier to code if we rewrite it like this:

First we use the product rule, $p(x,y) = p(y|x)p(x)$ to write the above as,

$$-\sum\limits_{x \in X} \sum\limits_{y \in Y}p(y|x)p(x) \log_2 p(y|x)$$

Then we note that $p(x)$ can move out of the inner summation, as it only depends on $x$:

$$-\sum\limits_{x \in X} p(x) \sum\limits_{y \in Y}p(y|x) \log_2 p(y|x)$$

So another way of writing the conditional entropy is "the expectation, over X, of the entropy of Y, given X".

Sort of like this (if it helps you remember),
$$
\mathbb{E}_X \Big[H(Y|X)\Big]
$$

This means you could write you conditional entropy function as:

1. loop over all the unique values in Xdata. Hint, try: `for v in Xdata.unique():`
2. For each one find the probability of the series having that value `np.mean(Xdata==v)`
3. Compute the entropy in ydata for just the rows in which Xdata is that value (`v`). You could do this by calling your `compute_entropy` method, with just the relevant values of ydata: `ydata[Xdata==v]`.
4. For each of the above compute the product of the probability you calculated in (2) and the entropy in (3), and sum them all, to get the expected entropy.
5. Return this sum.

---



### Exercise 1.2

Write a function to compute the conditional entropy,

In [ ]:
def compute_conditional_entropy(Xdata,ydata):
  """Return the Conditional entropy of ydata given the series (aka 'column') Xdata

   - Xdata, a series (i.e. a single column) from a pandas dataframe, e.g. X['long petal']
   - ydata, the associated labels

  returns the conditional entropy H(Y|X)
  """
  #answer here
  return #?

---

In [ ]:
testdata = pd.Series([False,True,True,True,False,False,True,False,False,False,True,True])
testlabels = pd.DataFrame(['red','blue','blue','blue','red','red','yellow','red','red','red','yellow','orange'])

#this testdata is 'false' when the labels were red, and true otherwise:
#so if false the entropy will be zero; if true, the entropy will be:
#3 blue, 2 yellow, 1 orange
#-(3/6 * np.log2(3/6) + 2/6 * np.log2(2/6) + 1/6 * np.log2(1/6))
#1.4591479170272448
#so the conditional entropy will be 0.5 * 0 + 0.5 * 1.45915 = 0.72957.

assert abs(compute_conditional_entropy(testdata,testlabels)-0.72957)<0.0001


Here we use the two methods you have written to compute the mutual information (information gain) for each of the features.

---

### Exercise 1.3

Can you finish off this method, what is missing?

Hint:
- We're computing $H(Y) - H(Y|X)$.
- $H(Y)$ is stored in `H_Y`.
- $H(Y|X)$ can be computed using `compute_conditional_entropy(Xdata[col],ydata)`.

In [ ]:
def compute_mutual_infos(Xdata,ydata):
  """
  For each column in Xdata, compute the mutual information with ydata
  (also known as information gain):

     H(Y) - H(Y|X_column)

  returns a pandas series with the mutual information for each column.
  """
  mutual_infos = {}
  H_Y = compute_entropy(ydata)
  for col in Xdata.columns:
    mutual_infos[col] = #answer here
  return pd.Series(mutual_infos)

compute_mutual_infos(X,y)

We now need to build the tree, at each split, we will pick the feature that has the greatest information gain:

In [ ]:
def build_tree(Xdata,ydata,depth=np.inf):
  mutual_infos = compute_mutual_infos(Xdata,ydata)
  col = mutual_infos.idxmax()
  if (mutual_infos[col]==0) or (depth<=0):
    return ydata.value_counts().idxmax()[0]

  tree = {'split':col}
  for v in Xdata[col].unique():
    subtree = build_tree(Xdata[Xdata[col]==v],ydata[Xdata[col]==v],depth-1)
    tree[v] = subtree

  return tree

#just using every other data point to train the tree. Max depth set to 2, try also max depth=1.
tree = build_tree(X[::2],y[::2],2)
print(tree)

In [ ]:
def predict(tree,x):
  subtree = tree[x[tree['split']]]
  if isinstance(subtree, str):
    return subtree
  else:
    return predict(subtree,x)

predictions = []
for i,xtest in X[1::2].iterrows():
  predictions.append(predict(tree,xtest))

print("Accuracy: %0.2f%%" % (100*np.mean(predictions==y[1::2]['class'].values)))

## sklearn's decision tree

Let's compare this tree with that generated with sklearn.

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2)
clf = clf.fit(X[::2], y[::2])

We can see what sort of tree sklearn creates using its 'tree' methods. The plot is somewhat unclear, but one can (after a bit of thinking) see that this matches our own tree:

In [ ]:
from sklearn import tree
tree.plot_tree(clf,feature_names=X.columns,class_names=y['class'].unique())

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y[1::2], clf.predict(X[1::2]))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=clf.classes_)
disp.plot()

# Part 2: Bootstrapping / Bagging

In the second half of this notebook we will look at applying the bootstrap sampling approach to a simple (one dimensional) regression problem.



## Bootstrapping applied to simple linear regression

We start with just a linear (1st-order "y = mx+c") fit to the data. Obviously we could use various library functions, but I'll try to keep the implementation within this notebook so you can see each step.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#our synthetic dataset
xvals = np.array([1.2,1.7,2.4,3.3,4.1,4.8,5.6,7.2,8.4,9.1])
yvals = np.array([3.2,3.7,4.2,4.6,4.9,5.1,5.6,5.9,6.1,6.4])

def fit_parameters(x,y):
  #Ordinary Least Squares fit to a 1st order polynomial
  # (straight-line).
  #Returns the two parameters as a numpy array
  # [gradient, offset]
  X = np.c_[x,np.ones_like(x)]
  return np.linalg.inv(X.T@X) @ X.T @ y

w = fit_parameters(xvals,yvals)
plt.plot(xvals,yvals,'x')
plt.plot([0,10],w[0]*np.array([0,10])+w[1])

---

### Exercise 2.1

We want to find out the uncertainty (given this model) on the range of possible predictions. To do this we will use bootstrapping and aggregation (bagging).

Complete the function below, you will need to:
1. Loop 1000 times...
2. For each iteration, select randomly *with replacement* ten of the values in x and y. Tip `np.random.choice(len(x),len(x))` might be useful!
3. For this selection compute $w$ using the `fit_parameters` method.
4. Add $w$ as another row in a matrix recording all the $w$s computed.
5. At the end of the loop there should be a $1000 \times 2$ matrix of values of $w$. Return this matrix.

In [ ]:
def bootstrap(x,y):
  #answer here
  return #?

---

It can be informative to plot the distribution over $w$. This gives a sense of what sort of distribution it might be, etc.

In [ ]:
ws = bootstrap(xvals,yvals)
plt.scatter(ws[:,0],ws[:,1],1)
plt.xlabel('Gradient')
plt.ylabel('Offset')

We can also plot the predictions on our normal xy graph. I'll just plot out 10% of them so the graph isn't overwhelmed. I've also drawn in the 95% credible interval for the predictions.

In [ ]:
def plotCIs(x,y,ws):
  testx = np.linspace(0,10,100)

  #I generate predictions for a grid of points, to allow us to draw the CIs.
  preds = ws[:,0:1]*testx[None,:]+ws[:,1:2]
  preds = np.sort(preds,0)
  plt.plot(testx,preds[25,:],'b--')
  plt.plot(testx,preds[500,:],'b-')
  plt.plot(testx,preds[975,:],'b--')

  #plot the raw data
  plt.plot(x,y,'xr')

  #plot 10% of the predictions
  for w in ws[::10]:
    plt.plot([0,10],[w[1],w[1]+w[0]*10],'k-',alpha=0.05)
plotCIs(xvals,yvals,ws)

## Decision trees and Bagging

Bagging helps when applied to the output of models or algorithms that are "unstable", e.g. decision trees, neural networks: These have high variance, but low bias. Linear regression on the other hand has fairly low variance (look how close the different results are above -- it is less likely to be sensitive to resampling).

We will now use bagging with simple 1-deep decision trees.

### Decision trees for a continuous input.

We have largely concerned ourselves with binary features. But if you remember in the lecture we briefly touched on continuous inputs and continuous outputs. The purity metric for continuous outputs is the expected variance; and to handle the input one selects a threshold for that feature that minimises the expected variances. The code below implements this:

In [ ]:
def get_split(x,y):
  """This method finds the threshold value that minimises the weighted
  sum of variances between the two sub-nodes. It returns this threshold
  and the means of the two sub-nodes.
  """

  minimum_var = np.inf
  minimum_threshold = np.NaN
  for threshold in np.linspace(0,10,100):
    weighted_sum_of_variance = np.var(y[y<threshold])*np.sum(y<threshold)+np.var(y[y>=threshold])*np.sum(y>=threshold)
    if weighted_sum_of_variance<minimum_var:
      minimum_var = weighted_sum_of_variance
      minimum_threshold = threshold
  return minimum_threshold, np.mean(y[x<minimum_threshold]), np.mean(y[x>=minimum_threshold])

In [ ]:
param  = get_split(xvals,yvals)
print(param)
plt.plot([0,param[0],param[0],10],[param[1],param[1],param[2],param[2]],'k-')
plt.plot(xvals,yvals,'x')

Two things to note:
1. The one-deep decision tree doesn't really seem to do a great job at prediction in this case.
2. It is probably going to be quite sensitive to resampling.

### Exercise 2.2.

Try writing a method that resamples 1000 times, with replacement, the points, and then for each resampled set, computes a new split (using the `get_split` method). Put all the parameters into a $1000 \times 3$ array. It is 3 wide as it includes the value of x where the tree splits the data, and includes the values on either side (left and right). Save in [splitLocation, leftValue, rightValue] order.

In [ ]:
def bootstrap_trees(x,y):
  """Run the bootstrap algorithm to make splits on bootstrap subsets of the data,
  then fit the 1-deep decision tree, using the get_split method. Return a numpy
  array of 1000x3 values, with the first column equal to the split points, the
  second column by the mean on the left of the split, and the third column the
  mean of the right of the split."""

  #answer here
  return #?

In [ ]:
parameters = bootstrap_trees(xvals,yvals)

In [ ]:
# Plot these predictions
for param in parameters:
  plt.plot([0,param[0],param[0],10],[param[1],param[1],param[2],param[2]],'k-',alpha=0.02)

#here we plot the 95% CI:
pred_x = np.linspace(0,10,100)
v = [np.nanmean(np.r_[parameters[p<parameters[:,0],1],parameters[p>=parameters[:,0],2]]) for p in pred_x]
lowerCI = [np.nanpercentile(np.r_[parameters[p<parameters[:,0],1],parameters[p>=parameters[:,0],2]],2.5) for p in pred_x]
upperCI = [np.nanpercentile(np.r_[parameters[p<parameters[:,0],1],parameters[p>=parameters[:,0],2]],97.5) for p in pred_x]
plt.plot(pred_x,v,'b-')
plt.plot(pred_x,lowerCI,'b--')
plt.plot(pred_x,upperCI,'b--')
plt.plot(xvals,yvals,'xr')

Argubly linear regression was a better model choice; however the mean of the ensemble is better than the single decision tree (feel free to check this!) and the model also provides uncertainty estimatation compared to a single tree.

# Part 3: Random Forest

We finally will apply what we have learnt to build a random forest.

This dataset is already very well separated with a simple decision tree, so a random forest is unlikely to help. We include the approach here purely for demonstrating how random forest works. Random forest is more effective where there is a greater risk of overfitting, and/or where many more features are available.


## Alternative tree-building method with subspace sampling

This method creates a tree as before, but at each node we only consider a random subset of the features. You can change the number of features chosen by changing `n=2` to, e.g. `n=1` in this line: `mutual_infos.sample(n=2)`.

In [ ]:
def build_tree_with_subspace_sampling(Xdata,ydata,depth=np.inf):
  """
  This method builds a decision tree, but at each node, we only keep 2 of the
  four features. This is called "subspace sampling".
  """
  mutual_infos = compute_mutual_infos(Xdata,ydata)
  mutual_infos = mutual_infos.sample(n=2) #just keep two of the four features.
  col = mutual_infos.idxmax()
  if (mutual_infos[col]==0) or (depth<=0):
    return ydata.value_counts().idxmax()[0]

  tree = {'split':col}
  for v in Xdata[col].unique():
    subtree = build_tree(Xdata[Xdata[col]==v],ydata[Xdata[col]==v],depth-1)
    tree[v] = subtree

  return tree

#just using every other data point to train the tree. Max depth set to 2.
build_tree_with_subspace_sampling(X[::2],y[::2],2)

## Building the random forest

Let's now build our forest. Make lots of trees using the subspace sampling. Here we make 50 different trees, each one might be different from the others as at each branch of the tree, we could be using a different subset of features. A given, individual tree, will therefore probably be worse on the test data than the standard decision tee created above, however by making them have less correlation with each other, we are able to reduce our overall error on the test set. This seems somewhat counter-intuitive!

In [ ]:
#Create 50 trees by calling build_tree_with_subspace_sampling
trees = []
for i in range(50):
  trees.append(build_tree_with_subspace_sampling(X[::2],y[::2],2))

#make predictions on each test item (the even items in the datasets)
predictions = []
for i,xtest in X[1::2].iterrows():
  singletree_predictions = []
  for tree in trees: #we record the predictions from all the individual trees...
    singletree_predictions.append(predict(tree,xtest))
  #we get the most popular prediction from all the trees in the random forest...
  predictions.append(max(set(singletree_predictions), key=singletree_predictions.count))

We can now compute the accuracy by looking at how many of our predictions matched the true label.

In [ ]:
print("Accuracy: %0.2f%%" % (100*np.mean(predictions==y[1::2]['class'].values)))

To explore the random forest approach a little more,
- set the max-depth to 1
- use 500 trees (to reduce the variability)
- reduce the subspace sampling to just pick one feature instead of two.

How does the accuracy compare to the single decision tree (**with depth=1**)? Does the random forest do better than the single tree? (I've not included the answer below, maybe discuss with friends!).


# Answers


### Exercise 1.1

In [ ]:
def compute_entropy(ydata):
  """
  Returns the entropy of the labels in ydata (a pandas series) in bits.
  """
  p = ydata.value_counts()/len(ydata)
  return -np.sum(p*np.log2(p))

### Exercise 1.2

In [ ]:
def compute_conditional_entropy(Xdata,ydata):
  """Return the Conditional entropy of ydata given the series (aka 'column') Xdata

   - Xdata, a series (i.e. a single column) from a pandas dataframe, e.g. X['long petal']
   - ydata, the associated labels

  returns the conditional entropy H(Y|X)
  """
  sum_over_x = 0
  for v in Xdata.unique():
    px = np.mean(Xdata==v)
    sum_over_x += px*compute_entropy(ydata[Xdata==v])
  return sum_over_x

### Exercise 1.3

In [ ]:
def compute_mutual_infos(Xdata,ydata):
  """
  For each column in Xdata, compute the mutual information with ydata
  (also known as information gain):

     H(Y) - H(Y|X_column)

  returns a pandas series with the mutual information for each column.
  """
  mutual_infos = {}
  H_Y = compute_entropy(ydata)
  for col in Xdata.columns:
    mutual_infos[col] = H_Y - compute_conditional_entropy(Xdata[col],ydata)
  return pd.Series(mutual_infos)

compute_mutual_infos(X,y)

### Exercise 2.1

In [ ]:
def bootstrap(x,y):
  ws = []
  for it in range(1000):
    indices = np.random.choice(len(x),len(x))
    w = fit_parameters(x[indices],y[indices])
    ws.append(w)
  return np.array(ws)

### Exercise 2.2

In [ ]:
def bootstrap_trees(x,y):
  """Run the bootstrap algorithm to make splits on bootstrap subsets of the data,
  then fit the 1-deep decision tree, using the get_split method. Return a numpy
  array of 1000x3 values, with the first column equal to the split points, the
  second column by the mean on the left of the split, and the third column the
  mean of the right of the split."""

  parameters = []
  for it in range(1000):
    indices = np.random.choice(len(x),len(x))
    split, left_mean, right_mean = get_split(x[indices],y[indices])
    parameters.append([split,left_mean,right_mean])
  return np.array(parameters)